## Наивный байес без учета стоп слов

In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("../data/desc.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [3]:
#импорт последнего датасета (со спарсенными)
#df = pd.read_csv("../data/mergedcleared0407.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
df = pd.read_csv("../data/lemma.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [4]:
df['label'] = df['label'].str.lower()
df['label'] = df['label'].str.strip()
df['label'] = df['label'].str.replace(r'\s+', ' ')
df['label'] = df['label'].str.replace(r'[^\w\s]+', ' ')
df['label'] = df['label'].str.replace(r'\d+', ' ')
df['label'] = df['label'].str.strip()
df['label'] = df['label'].str.strip()

C:\Users\yeril\AppData\Local\Temp\ipykernel_9332\3288965867.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['label'] = df['label'].str.replace(r'\s+', ' ')
C:\Users\yeril\AppData\Local\Temp\ipykernel_9332\3288965867.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['label'] = df['label'].str.replace(r'[^\w\s]+', ' ')
C:\Users\yeril\AppData\Local\Temp\ipykernel_9332\3288965867.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['label'] = df['label'].str.replace(r'\d+', ' ')


In [6]:
df.shape

(261080, 2)

In [5]:
#убираем классы, которых меньше 10 экземпляров
counts = df.value_counts('id')
df_count = pd.DataFrame(data={'id': counts.index, 'count': counts.values})
df = df[~df['id'].isin(df_count[df_count['count']<10]['id'])]

In [7]:
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

### Лемматизация текста 

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [6]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [32]:
subdf = df#.sample(5000)
lemma = list()
for each in tqdm(subdf['label']):
    lemma.append(' '.join(lemmatize(each)))
subdf['lemma'] = lemma
subdf

100%|████████████████████████████████████████████| 261080/261080 [15:22<00:00, 283.02it/s]


,id,label,lemma
0,8425310000,лебедка с приводом от электрического двигателя...,лебёдка с привод от электрический двигатель бы...
1,8425310000,лебедки для пассаж элект лифта мод kone minisp...,лебёдка для пассаж элект лифт мода kone minisp...
2,8425310000,лебедки с приводом от электрического двигателя...,лебёдка с привод от электрический двигатель а ...
3,8425310000,части лифтового оборудования для сервисного об...,часть лифтовый оборудование для сервисный обсл...
4,8425310000,лебедки для пассаж элект лифта мод kone monosp...,лебёдка для пассаж элект лифт мода kone monosp...
...,...,...,...
261742,1207409000,семена кунжута sesamum indicum очищенные готов...,семя кунжут sesamum indicum очистить готовый к...
261743,1207409000,кунжут семена кунжута очищенные не дробленные ...,кунжут семя кунжут очистить не дробить не для ...
261744,1207409000,семена кунжута,семя кунжут
261745,1207409000,семена кунжута дробленые или недробленые,семя кунжут дроблёный или недроблёный


In [33]:
subdf.to_csv('../data/lemma.csv', sep=';', columns=['id', 'lemma'], index=False, header=False)

### Выбираем модели наивного байеса 

In [7]:
#импорт наивного байеса
from sklearn.naive_bayes import ComplementNB, MultinomialNB, BernoulliNB, CategoricalNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
#from sklearn.model_selection import GridSearchCV, cross_val_score

In [7]:
#делаем вокабуляр для векторов предложений

vect = CountVectorizer(binary=True)
vect.fit(df['label'])
len(vect.vocabulary_)

120388

In [5]:
#unlemmatized
#делаем вокабуляр для веркоров предложений
#vect = CountVectorizer(binary=True)
#vect.fit(df['label'])
#len(vect.vocabulary_)

254935

In [8]:
#делим на тест и трэйн
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df['id'], shuffle=True)

In [9]:
#получаем векторы предложений
X_train = vect.transform(df_train['label'])
X_test = vect.transform(df_test['label'])

In [10]:
#обучаем всю эту ситуацию
#дико прожорливая до оперативки и очень медленная модель при обучении
clf = ComplementNB().fit(X_train, df_train['id'])

In [42]:
#сохраним 4,5 ГБ модель (чтоб по 100 раз не обучать)
#
import pickle
filename = '../../../finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# Грузим так
#loaded_model = pickle.load(open(filename, 'rb'))

In [11]:
#смотрим предсказания на тесте
predicts = clf.predict(X_test)
print(classification_report(df_test['id'], predicts))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  0207149909       0.78      1.00      0.88        21
  0304999800       0.60      1.00      0.75        18
  0305542000       0.74      1.00      0.85        14
  0305549000       0.82      0.74      0.78        19
  0305599000       0.75      0.82      0.78        22
  0604909900       0.58      0.90      0.70        21
  0701905000       0.93      1.00      0.96        26
  0703200000       0.68      0.95      0.79        20
  0711400000       0.69      0.75      0.72        12
  0711590000       1.00      0.50      0.67         4
  0712909000       1.00      0.71      0.83        21
  0713339000       0.93      0.88      0.90        16
  0801110000       0.82      0.92      0.87        25
  0802310000       0.72      0.39      0.51        33
  0802320000       0.62      1.00      0.77        51
  0803901000       0.95      1.00      0.97        18
  0804300009       1.00      0.79      0.88        19
  0804500001       0.75    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
#Как пользоваться:
#формируем массив текстовых запросов
#лемматизируем 
#Передаем методу трансформ список текстовых описаний, по которым хотим получить классы. 
#Получаем векторы предложений...
request = ['нешелушенные грецкие орехи в коробках', 'шины автомобильные радиальные новые', 'трубы полипропиленовые', 'презервативы'] #например
lem = list()
for each in request:
    lem.append(' '.join(lemmatize(each)))
txt_to_vector = tfvect.transform(lem)

In [14]:
#скармливаем векторы модели, получаем предсказания
result = clf.predict(txt_to_vector)

In [15]:
#Читаем, что получили

for each in result:
    #print(each[:4])
    print(each, dscr[dscr['id'] == each[:4]].iloc[0]['label'])

0802320000 Прочие орехи, свежие или сушеные, очищенные от скорлупы или неочищенные, с кожурой или без кожуры
4011100009 Шины и покрышки пневматические резиновые новые
5402340000 Нити комплексные синтетические (кроме швейных ниток), не расфасованные для розничной продажи, включая синтетические мононити линейной плотности менее 67 дтекс
4014100000 Изделия гигиенические или фармацевтические (включая соски) из вулканизованной резины, кроме твердой резины, с фитингами из твердой резины или без них


In [44]:
del(clf)

### Tf-Idf 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfvect = TfidfVectorizer()#norm=None, max_df=0.8, max_features=50000, decode_error='replace')
tfvect.fit(df['label'])
len(tfvect.vocabulary_)

120388

In [10]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df['id'], shuffle=True)
X_train = tfvect.transform(df_train['label'])
X_test = tfvect.transform(df_test['label'])
clf = ComplementNB()

In [11]:
clf.fit(X_train, df_train['id'])

ComplementNB()

In [12]:
predicts = clf.predict(X_test)
print(classification_report(df_test['id'], predicts))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  0207149909       0.78      1.00      0.88        21
  0304999800       0.74      0.94      0.83        18
  0305542000       0.78      1.00      0.88        14
  0305549000       0.78      0.74      0.76        19
  0305599000       0.83      0.91      0.87        22
  0604909900       0.64      0.86      0.73        21
  0701905000       0.87      1.00      0.93        26
  0703200000       0.61      1.00      0.75        20
  0711400000       0.75      0.25      0.38        12
  0711590000       1.00      0.50      0.67         4
  0712909000       1.00      0.62      0.76        21
  0713339000       1.00      1.00      1.00        17
  0801110000       0.70      0.84      0.76        25
  0802310000       0.91      0.30      0.45        33
  0802320000       0.57      1.00      0.72        51
  0803901000       0.95      1.00      0.97        18
  0804300009       0.94      0.79      0.86        19
  0804500001       0.82    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Берт + логистическая регрессия, полиномиальные признаки, нормализации, опорные вектора

In [47]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")


C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
#Делим на 100 частей, чтобы хватило оперативки

chunks = np.array_split(df, 100)
s_embgs = np.array([1])
i = 1
for chunk in chunks:
    print(f"Getting embgs for {i} chunk, {len(chunk)} points")
    sentences = chunk['label'].tolist()
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    
    if (len(s_embgs) == 1):
        s_embgs = sentence_embeddings.detach().numpy()
    else:
        s_embgs = np.concatenate((s_embgs, sentence_embeddings.detach().numpy()), axis=0)
    i += 1

Getting embgs for 1 chunk, 2611 points


KeyboardInterrupt: 

In [10]:
import re
words = ['python 3', 'python3', '1something', '2', '434', 'python 35', '1 ', ' 232']
for word in words:
    print("'{}' => '{}'".format(word, re.sub(r'\b\d+\b', '', word)))

'python 3' => 'python '
'python3' => 'python3'
'1something' => '1something'
'2' => ''
'434' => ''
'python 35' => 'python '
'1 ' => ' '
' 232' => ' '


In [11]:
re.sub(r'\b\d+\b', '', 'всем3 привет как 543 дела')

'всем3 привет как  дела'

In [13]:
re.sub(r'\d+', '', 'всем3 привет как 543 дела')

'всем привет как  дела'